# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298596818357                   -0.85    5.2         
  2   -8.300218715872       -2.79       -1.25    1.0    117ms
  3   -8.300436445060       -3.66       -1.89    3.1    154ms
  4   -8.300459810348       -4.63       -2.72    1.6    125ms
  5   -8.300463051772       -5.49       -2.98    7.9    292ms
  6   -8.300463786786       -6.13       -3.15    2.4    133ms
  7   -8.300464205576       -6.38       -3.29    1.8    134ms
  8   -8.300464408378       -6.69       -3.42    1.0    108ms
  9   -8.300464607483       -6.70       -3.71    3.1    177ms
 10   -8.300464628237       -7.68       -3.86    2.1    212ms
 11   -8.300464638341       -8.00       -4.09    1.5    136ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64153495316                   -0.70    5.5         
  2   -16.67459136983       -1.48       -1.14    1.4    296ms
  3   -16.67919581629       -2.34       -1.88    3.8    364ms
  4   -16.67927609939       -4.10       -2.73    3.4    332ms
  5   -16.67928563000       -5.02       -3.08    4.6    449ms
  6   -16.67928619021       -6.25       -3.48    2.0    294ms
  7   -16.67928621575       -7.59       -3.98    2.8    319ms
  8   -16.67928622150       -8.24       -4.57    2.0    310ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32591968258                   -0.56    6.6         
  2   -33.33281813438       -2.16       -1.00    1.1    993ms
  3   -33.33405739056       -2.91       -1.73    4.6    1.33s
  4   -33.33427052303       -3.67       -2.62    2.5    1.09s
  5   -33.33694096267       -2.57       -2.48    5.2    1.50s
  6   -33.33694123947       -6.56       -2.48    8.0    1.41s
  7   -33.33693665545   +   -5.34       -2.52    1.1    954ms
  8   -33.33690697002   +   -4.53       -2.62    3.1    1.10s
  9   -33.33690871900       -5.76       -2.66    1.1    954ms
 10   -33.33691474430       -5.22       -2.72    1.0    907ms
 11   -33.33692567697       -4.96       -2.81    1.1    918ms
 12   -33.33693291107       -5.14      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298284961376                   -0.85    5.2         
  2   -8.300249096636       -2.71       -1.58    1.0   86.0ms
  3   -8.300386688301       -3.86       -2.34    2.5    136ms
  4   -8.300327860323   +   -4.23       -2.18    3.6    151ms
  5   -8.300463795332       -3.87       -3.38    1.0   87.6ms
  6   -8.300464515050       -6.14       -3.67    3.1    146ms
  7   -8.300464636278       -6.92       -4.13    1.4   94.9ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32686227362                   -0.56    7.1         
  2   -33.31648815294   +   -1.98       -1.27    1.2    843ms
  3   -15.65809645913   +    1.25       -0.44    6.2    1.82s
  4   -33.31731315425        1.25       -1.61    5.2    1.57s
  5   -33.23263016304   +   -1.07       -1.36    3.9    1.35s
  6   -31.36196005845   +    0.27       -0.92    4.1    1.39s
  7   -33.33412872087        0.29       -2.31    4.6    1.34s
  8   -33.33536141533       -2.91       -2.21    2.9    1.36s
  9   -33.33661666055       -2.90       -2.72    2.1    1.06s
 10   -33.33671226701       -4.02       -2.84    1.5    908ms
 11   -33.33691760658       -3.69       -3.18    3.4    1.11s
 12   -33.33693904156       -4.67      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.